In [28]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import random
import numpy as np
import pandas as pd
import re

In [29]:
def setup_driver():
    brave_path = r"C:\Program Files\BraveSoftware\Brave-Browser\Application\brave.exe"
    service = Service(ChromeDriverManager().install())
    options = Options()
    options.binary_location = brave_path
    driver = webdriver.Chrome(service=service, options=options)
    return driver

In [30]:
def get_page_source(url, driver):
    driver.get(url)
    time.sleep(10)
    wait = WebDriverWait(driver, 10)
    return driver.page_source

In [31]:
def extract_articles_jornal_noticias(soup):
    divs_itc001 = soup.find_all(lambda tag: (
        tag.name == "div" and 
        tag.get("data-instance") == "i01" and 
        "ITC001" in tag.get("class", []) and 
        "light" in tag.get("class", [])
    ))
    divs_hc002 = soup.find_all(lambda tag: (
        tag.name == "div" and 
        "HC002" in tag.get("class", []) and 
        "light" in tag.get("class", [])
    ))
    divs_pc001 = soup.find_all(lambda tag: (
        tag.name == "div" and 
        tag.get("data-instance") == "i01" and 
        "PC001" in tag.get("class", []) and 
        "light" in tag.get("class", [])
    ))
    divs_hc002data = soup.find_all(lambda tag: (
        tag.name == "div" and 
        tag.get("data-instance") == "i02" and 
        "HC002" in tag.get("class", []) and 
        "light" in tag.get("class", [])
    ))
    divs_itc002 = soup.find_all(lambda tag: (
        tag.name == "div" and 
        tag.get("data-instance") == "i04" and 
        "ITC002" in tag.get("class", []) and 
        "light" in tag.get("class", [])
    ))
    
    print(f"HC002 light none found: {len(divs_hc002data)}")
    print(f"ITC001 light none found: {len(divs_itc001)}")
    print(f"PC001 light corners-square none hover-scale found: {len(divs_pc001)}")
    print(f"ITC002 light corners-square none found: {len(divs_itc002)}")
    print(f"HC002 light none found: {len(divs_hc002data)}")

    all_divs = divs_hc002 + divs_itc001 + divs_pc001 +  divs_itc002 + divs_hc002data

    articles = []
    for div in all_divs:
        #print(div)
        a_tag = div.find("a")
        h2title, category = None, None
        h2title = div.find("h2",class_="title")
        category = div.find("div", class_="info")

        if a_tag:

            h2title = h2title.get_text(strip=True)
            category = category.get_text(strip=True)
            link = a_tag["href"]
            
            full_link = f"https://www.jn.pt{link}"

            articles.append({"title": h2title, "link": full_link, "category": category})

        else:
            continue
            
    return articles

In [46]:
def scroll_and_find_element(driver,locator,scroll_amount=500):
    wait = WebDriverWait(driver, 10)
    scroll_position = 0
    print(f"Entro aqui também")
    while True:
        try:
            print(f"Entro aqui também 2")
            element = wait.until(EC.presence_of_element_located(locator))
            print(f"Encontrei o element -> {element}")
            return element
        except:
            print(f"Entro aqui também 3 ")
            driver.execute_script(f"window.scrollTo(0,{scroll_position})")
            scroll_position += scroll_amount
            time.sleep(20)

            if scroll_position > driver.execute_script("return document.body.scrollHeight"):
                print("End of file reached, it didnt find the element")
                return None

In [44]:
def extract_full_article(article_url, driver):
    """Visit an article URL and extract the full text content."""
    page_source = get_page_source(article_url, driver)
    soup = BeautifulSoup(page_source, "html.parser")

    paragraphs = soup.find_all("p")
    return "\n".join([p.get_text(strip=True) for p in paragraphs])

In [52]:
def main():
    url = "https://www.jn.pt/"
    driver = setup_driver()

    get_page_source(url, driver)
    driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[2]/div[2]/div[2]/button[1]').click()
    time.sleep(1)
    
    total_height = driver.execute_script("return document.body.scrollHeight")
    quarter_height = total_height / 4
    current_scroll_position = 0

    for _ in range(4):
        current_scroll_position += quarter_height
        driver.execute_script(f"window.scrollTo(0, {current_scroll_position});")
        time.sleep(5) #Add a small delay, adjust as needed

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")

    time.sleep(1)
    driver.execute_script(f"window.scrollTo(0, 0);","")
    time.sleep(1)

    try:
        print(f"Estou a entrar")
        element_crise_politica = scroll_and_find_element(driver,(By.XPATH, '//*[@id="crise-politica"]/div/div/div/div[1]/div/a'))
        print(f"Conteudo da funcao -> {element_crise_politica}")
        driver.execute_script("arguments[0].scrollIntoView();", element_crise_politica)
        time.sleep(2)
        element_crise_politica.click()
        time.sleep(10)
        print(f"Crise Politica button pressed.")
    except Exception as e:
        print(f"Crise Politica couldnt be pressed")

    current_scroll_position = 0
    
    for _ in range(4):
        current_scroll_position += quarter_height
        driver.execute_script(f"window.scrollTo(0, {current_scroll_position});")
        time.sleep(random.randint(1,2))

    articles = extract_articles_jornal_noticias(soup)


    # for article in articles:
    #     article["article_content"] = extract_full_article(article["link"], driver)

    #     print(f"Title: {article['title']}")
    #     print(f"Link: {article['link']}")
    #     print(f"Content Inside Div: {article['div_content']}")
    #     print(f"Full Article Content:\n{article['article_content'][:500]}...")
    #     print("-" * 100)

    driver.quit()

    df = pd.DataFrame(articles)

    df = df.drop_duplicates(subset=['title'],ignore_index=True)

    df_politica = df[df['category'].isin(['Politica','Política','politica']) ]
    display(df_politica)


    
if __name__ == "__main__":
    main()


Estou a entrar
Entro aqui também
Entro aqui também 2
Encontrei o element -> <selenium.webdriver.remote.webelement.WebElement (session="6f648a1250ec320f9651f0d2ccbdb6b4", element="f.4A41CC3EDD1C367C5BF546740955C48A.d.9B97BFB76C0528D20246B54CF2CD5C16.e.621")>
Conteudo da funcao -> <selenium.webdriver.remote.webelement.WebElement (session="6f648a1250ec320f9651f0d2ccbdb6b4", element="f.4A41CC3EDD1C367C5BF546740955C48A.d.9B97BFB76C0528D20246B54CF2CD5C16.e.621")>
Crise Politica button pressed.
HC002 light none found: 11
ITC001 light none found: 1
PC001 light corners-square none hover-scale found: 9
ITC002 light corners-square none found: 68
HC002 light none found: 11


,title,link,category
33,Licença parental e acesso de estrangeiros ao S...,https://www.jn.pt/6220613528/licenca-parental-...,Política
35,"Montenegro convicto de que Portugal ""vai tirar...",https://www.jn.pt/6202629582/montenegro-convic...,Política
